In [2]:
import pandas as pd
from pandas_datareader import data
from datetime import datetime
from dateutil.relativedelta import relativedelta
import requests
from enum import Enum
import pprint

pp = pprint.PrettyPrinter(indent=1)

ALPHA_VANTAGE_API_KEY = '111111111111'


# Notes
## Key indicators
1. Gross profit margin = (revenue - cost) / revenue * 100
2. EBITDA margin = EBITDA / revenue * 100
3. Expenditure as % of revenue = expenditure / revenue
4. Net current asset value = (total liabilities - current assets) / number of outstanding shares
5. Net working capital
6. Benjamin Graham valuation formula (3% risk free rate)
7. Earnings Power Value
8. Fair Value PE = Basic PE x [1 + (1 - Business Risk)] x [1 + (1 - Financial Risk)] x [1 + (1 - Earnings Visibility)]
  - Earnings growth rate
  - Dividend yield
  - Business risk
  - Financial risk
  - Earnings visibility
9. Reverse discounted cash flow (GIC dun like?)

In [3]:
class AlphaVantageFundamentalFunctions(Enum):
    INCOME_STATEMENT = 'INCOME_STATEMENT'
    BALANCE_SHEET = 'BALANCE_SHEET'
    CASH_FLOW = 'CASH_FLOW'
    EARNINGS = 'EARNINGS'
    COMPANY_OVERVIEW = 'OVERVIEW'
    LISTING_AND_DELISTING_STATUS = 'LISTING_STATUS'  # not frequently used
    EARNINGS_CALENDAR = 'EARNINGS_CALENDAR'
    IPO_CALENDAR = 'IPO_CALENDAR'


def getTicker(companyName):
    url = f'https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords={companyName}&apikey={ALPHA_VANTAGE_API_KEY}'
    response = requests.get(url=url)
    data = response.json()
    return [x.get('1. symbol') for x in data.get('bestMatches', [])]


def getFundamentalData(function: AlphaVantageFundamentalFunctions, ticker: str):
    url = f'https://www.alphavantage.co/query?function={function.value}&symbol={ticker}&apikey={ALPHA_VANTAGE_API_KEY}'
    r = requests.get(url)
    data = r.json()

    return data

def calculateGrossMargin(annualReport):
    revenue = int(annualReport.get('totalRevenue', '0'))
    cost = int(annualReport.get('costOfRevenue', '0'))
    return (revenue - cost) / revenue * 100

def calculateEBITDAMargin(companyOverview):
    ebitda = int(companyOverview.get('EBITDA', '0'))
    


In [5]:
companyOverview = getFundamentalData(AlphaVantageFundamentalFunctions.COMPANY_OVERVIEW, 'AAPL')



{'Symbol': 'AAPL',
 'AssetType': 'Common Stock',
 'Name': 'Apple Inc',
 'Description': "Apple Inc. is an American multinational technology company that specializes in consumer electronics, computer software, and online services. Apple is the world's largest technology company by revenue (totalling $274.5 billion in 2020) and, since January 2021, the world's most valuable company. As of 2021, Apple is the world's fourth-largest PC vendor by unit sales, and fourth-largest smartphone manufacturer. It is one of the Big Five American information technology companies, along with Amazon, Google, Microsoft, and Facebook.",
 'CIK': '320193',
 'Exchange': 'NASDAQ',
 'Currency': 'USD',
 'Country': 'USA',
 'Sector': 'TECHNOLOGY',
 'Industry': 'ELECTRONIC COMPUTERS',
 'Address': 'ONE INFINITE LOOP, CUPERTINO, CA, US',
 'FiscalYearEnd': 'September',
 'LatestQuarter': '2022-03-31',
 'MarketCapitalization': '2344749892000',
 'EBITDA': '130633998000',
 'PERatio': '23.56',
 'PEGRatio': '2.56',
 'BookVal

In [3]:
tickers = ['AAPL', 'AMZN']

start_date = (datetime.now() - relativedelta(years=5)).strftime("%Y-%m-%d")
end_date = datetime.now().strftime("%Y-%m-%d")

# Use pandas_reader.data.DataReader to load data
panel_data = data.DataReader(tickers, 'yahoo', start_date, end_date)


In [4]:
moving_averages = [50, 100, 200]
for tic in tickers:
    for ma in moving_averages:
        panel_data[f'ma{ma}', tic] = panel_data.Close[tic].rolling(
            window=ma).mean()


In [23]:
getTicker('tesla')

['TSLA', 'TL0.DEX', 'TL0.FRK', 'TSLA34.SAO']